# Web and Text Analytics - Sentiment Analysis

### Authors
- Francois Cubelier
- Lisa Bueres
- Romain Charles

## Introduction
This report showcases our results for sentiment analysis. We experimented different types of recurrent neural networks on this task: basic RNN, LSTM and GRU. We studied the effect of attention on these models as well as testing different word embeddings (GloVe, FastText and Word2Vec). Finally, document level embedding were also investigated (WME, Bert sentence, GloVe average and a task specific average).

### Sentiment Analysis
Sentiment analysis is the analysis of texts such as reviews in order to detect the polarity implied by the author. The polarity is either positive, negative or neutral.

## Methods

### RNNs

3 types of recurrent neural networks were tested: Base RNN, LSTM and GRUs. We also experimented with different hyperparameters: 
- The number of hidden layers: 1 or 2
- The size of the hidden state: 100 or 300
- The type of output layer after the main network: linear or MLP (for more details on the MLP, see models/RNN.py)

### Attention
We tested 2 types of attention based on scaled dot-product attention: 
- Last hidden layer attention: 
$Attention(Q, K, V) = softmax(QK^\top/\sqrt n)V$ where V = K = output sequence and Q is the last hiddent state
- Self-attention followed by last hiddent layer attention.
First we apply a self attention:
$Attention_1(Q, K, V) = V' = softmax(QK^\top/\sqrt n)V$ where V = K = Q = output sequence.
Then we apply the last hidden layer attention:
$Attention_2(Q', K', V') = softmax(Q'K'^\top/\sqrt n)V'$ where V' = K' = output sequence of self-attention and Q is the last hidden state of the RNN.


### Document level embeddings
There numerous document level embeddings in the literature.
In tested 4 methods:
- Average glove embeddings: a sentence is encoded by the average of its GloVe word embeddings.
- Task specific average: a sentece is encoded by the average of its word embeddings and the word embedding are trained from scratch for this task.
- WME: Document embedding based on the distance (based on Word Mover Distance) between the document and randomly generated documents.
- Sentence BERT: a pre-trained document level embedding based on BERT (all-MiniLM-L6-v2 model).

In order to predict the class from the document embedding, 2 types of model were tested: linear model and MLP.

## Experiments

### Experimental setup

For our experiments, we train our model with Adam optimizer ($\beta_1 = 0.9, \beta_2=0.999)$) with a lreaning rate of $0.001$. For experiment 1 to 4, a batch size of 32 was used and 128 for experiment 5. 
We used early stopping, the validation loss is monitored during training (on 10% of the initial training set) and the training is stopped if there were no improvement during the last t epochs (t=4 for Exp.1 and 6 for the others). Later, the number of iterations is also called the number of steps. The last model is kept and is typically overfitted.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Experiment 1: Hyperparameters
In this experiment, we compare RNN, LSTM and GRU over the IMDB datasets and examine the results by varying hyperparameters.

List hparams and explain.

In [ ]:
df_1 = pd.read_csv("results/exp1_all.csv")

#### Model
As can be seen by computing the mean of the test accuracy and the test loss, GRU and LSTM are better than RNN. GRU and LSTM have mean accuracies very close GRU being slighty better than LSTM, but LSTM is able to better reduce the loss in average. RNN performed around 30% less than the others for the accuracy and the the loss, which is a non negligible difference.

Both GRU and LSTM use memory while the simple RNN does not. This allows the two first to have more information than the RNN when performing the classification, explaining the difference in performance.

In [ ]:
df_1.groupby("Model Type").mean()[["Test Acc", "Test Loss"]]

In [ ]:
sns.barplot(x="Model Type", y="Test Loss", data=df_1)

#### Output Layer Type
The table below shows that linear layers are better than MLP layers, on both the accuracy and the loss. This can be due to the Multi-Layer Perceptron (MLP) leading to a more complex model, because of non linearity, with overfitting on the training set and thus reduced performance on the testing set, while the linear model is simpler and limited in overtfitting thanks to its structure not allowing too complex functions. Additionally, using a more complex model after the RNN may not force the RNN to learn a good representation of the sentence (not linearly separable).

In [ ]:
df_1.groupby("Output Layer Type").mean()[["Test Acc", "Test Loss"]]

#### Number of layers
As shown in the table below, adding one layer slighlty improves the average accuracy and decreases the loss. However it also increase the computation time (TODO: add numbers)

In [ ]:
df_1.groupby("Number of Layers").mean()[["Test Acc", "Test Loss"]]

#### Number of features in the hidden state
Increasing the number of features in the hidden state (hidden size) leads to slightly less performance. This is shown in the table below where the size goes from 100 to 300. The average difference is really tight as it is of around 2% for both the average accuracy and the average loss.

In [ ]:
df_1.groupby("Hidden Size").mean()[["Test Acc", "Test Loss"]]

#### Full results
This table allowing to compare the results for all the hyper-parameters simultaneously corroborates what has been said previously. However, the effect of the number of layers is not much meaningfull: increasing from 1 to 2 sometimes increases the performance, sometimes decreases it.
But this table brings new information: the best test accuracy, 0.88768, is reached by the GRU model, with linear layer, a hidden size of 100 and two layers. This configuration has also the fourth smallest loss, which upholds the qualification as the best in the table. Its number of steps is 5632, which is a quite decent amount. The minimal loss, 0.287417, is realized by a LSTM model, also linear with 100 features and 1 layer. The test accuracy is also the third better, thus making this combination of parameters a pretty good choice too. The number of steps reaches 6336 this time, meaning more computation time. Without distinguishing the model, the hyper parameters of those two combinations are the same.

TODO: c'est mieux +l'accuracy ou - de loss, ou une bonne combinaison des deux? :)

At the opposite, the worst performances are realized with the RNN, far from LSTM and GRU. The worst accuracy, 0.49788, is the RNN linear with a size of 300 and 2 layers, tight to the same configuration with MLP instead of linear. The highest loss, 0.722325, is achieved with the linear RNN, with a size of 300 and 1 layer. An fact worth noticing on the RNN is that, depending on the hyper-parameters, the test loss can be higher than the test accuracy, making those situations useless. This worst accuracy has a normal number of steps but the worst loss has a high number of 7040 steps.

We can see that the number of steps of LSTM is generally higher than the ones of GRU and RNN with values at 6336 or above up to 8448, while the RNN has the smallest at 4224 and GRU at 4928.

TODO: je peux dire que prendre du random donnerait des résultats quasi pareil ? XD
Experiment 1 complete results (TODO: in Appendix ?)

In [ ]:
df_1.groupby(["Model Type","Output Layer Type", "Hidden Size", "Number of Layers"], sort=True).max()[["Test Acc", "Test Loss", "step"]]

#### Validation Loss monitoring
Let us analyse the plot below. Increasing the number of steps has only a small effect on the RNN, decreasing the loss solely at the end, and not enough to consider the RNN as serious challenger. 
For LSTM and GRU, increasing the steps first decreases the loss, but increases it again, when exceeding around 2000 for GRU and 4000 for LSTM. In order to obtain the best results, the number of steps must thus be limited to stay close to the minimum loss for each. The minimal loss is seen to be achieved with LSTM, as said previously. LSTM is also more stable around its minimum, the area around being tighter.

In [ ]:
sns.lineplot(x="step", y="Val Loss", hue="Model Type", data=df_1)

#### Comparison of accuracy and loss
We can see on the plot below that high accuracy and small loss are achieved by both LSTM and GRU, present in the bottom right corner. GRU is seen here to always keep a high accuracy while LSTM is more spread. Sadly, the green points cover some orange ones, making it a bit harder to analyse.
Once again at the opposite, the RNN model keeps a high loss and a small accuracy. It is indeed present the most in the upper left corner. 
We can notice that GRU and RNN never overlap, while LSTM overlaps with the RNN but barely, and mostly overlaps with the GRU model.

TODO: y a des hyperparamètres à mentionner ici ?

In [ ]:
sns.scatterplot(x="Val Acc", y="Val Loss", hue="Model Type", data=df_1)

#### Distribution of the loss and the accuracy
Both loss and accuracy distributions for the three models looks like the normal distribution.
For the accuracy, we can see that the RNN is highly spread, with a mean of around 0.5, which is close to the value seen before. The mean used at the beginning ot the report thus hide some better results, but those are not significant. The accuracies of the LSTM and GRU model are at the opposite very tight around the mean, which was of 0.88.
For the loss, the one of the RNN model is moderately spread, but the mean is high. The losses of LSTM and GRU are a bit more spread, still moderately, but have their mean around better values: 0.45 for GRU and 0.38 for LSTM. As mentionned in the previous section about the scatterplot, LSTM and GRU overlap a lot, while RNN barely overlaps with the two.
The performances of LSTM and GRU are very close and this is visible here too. GRU reaches more often a better accuracy but LSTM can reach the same, while LSTM is also able to get a great part of its points in the smallest loss area. But the spread means that a small loss is not always guaranteed, leading to the configuration of hyper-parameters being even more important.


In [ ]:
sns.jointplot(x="Test Acc", y="Test Loss", hue="Model Type", data=df_1, kind="scatter")

### Conclusion of Experiment 1:
Overall, the best results are definitely obtained by GRU and LSTM. As the hyper parameters are the same for the smallest loss and the best accuracy, choosing between GRU and LSTM is more of a matter of computation time and preferences than anything else. 

## Experiment 2: Attention
In this experiment, we analyse the effect of adding an attention layer for the sentiment analysis.

In [ ]:
df_2 = pd.read_csv("results/exp2_all.csv")

In [ ]:
df_2.groupby("Model Type").mean()[["Test Acc", "Test Loss"]]

In [ ]:
df_2.groupby("Attention").mean()[["Test Acc", "Test Loss"]]

In [ ]:
sns.barplot(x="Attention", y="Test Loss", data=df_2)

In [ ]:
sns.lineplot(x="step", y="Val Loss", hue="Attention", data=df_2)

In [ ]:
df_2.groupby(["Model Type","Attention"], sort=True).max()[["Test Acc", "Test Loss", "step"]]

## Experiment 3: Word level embedding

In [ ]:
df_3 = pd.read_csv("results/exp3_all.csv")

In [ ]:
df_3.groupby("Embedding").mean()[["Test Acc", "Test Loss"]]

In [ ]:
sns.barplot(x="Embedding", y="Test Loss", data=df_3)

In [ ]:
df_3.groupby(["Model Type","Embedding"], sort=True).max()[["Test Acc", "Test Loss"]]

In [ ]:
sns.lineplot(x="step", y="Val Loss", hue="Embedding", data=df_3)

In [ ]:
sns.scatterplot(x="Val Acc", y="Val Loss", hue="Embedding", data=df_3)

In [ ]:
sns.jointplot(x="Test Acc", y="Test Loss", hue="Embedding", data=df_3, kind="scatter")

## Experiment 4: Document level embedding

In [ ]:
df_4 = pd.read_csv("results/exp4_all.csv")

In [ ]:
df_4.groupby("Model Type").mean()[["Test Acc", "Test Loss"]]

In [ ]:
sns.barplot(x="Model Type", y="Test Loss", data=df_4)

#### Ouput layer type
Here we can see that if some information can be extracted, it can be extracted mostly linearly.

In [ ]:
df_4.groupby("Output Layer Type").mean()[["Test Acc", "Test Loss"]]

In [ ]:
sns.lineplot(x="step", y="Val Loss", hue="Model Type", data=df_4)

In [ ]:
sns.scatterplot(x="Val Acc", y="Val Loss", hue="Model Type", data=df_4)

In [ ]:
sns.jointplot(x="Test Acc", y="Test Loss", hue="Model Type", data=df_4, kind="scatter")

In [ ]:
df_4.groupby(["Model Type","Output Layer Type","Embedding Size"], sort=True).mean()[["Test Acc", "Test Loss"]]

## Experiment 5: Performance on different datasets

In [ ]:
df_5 = pd.read_csv("results/exp5_all.csv")

In [ ]:
df_5.groupby("Dataset").mean()[["Test Acc", "Test Loss"]]

In [ ]:
sns.barplot(x="Dataset", y="Test Loss", data=df_5)

In [ ]:
sns.lineplot(x="step", y="Val Loss", hue="Dataset", data=df_5)

In [ ]:
df_5.groupby(["Dataset", "Model Type"], sort=True).max()[["Test Acc", "Test Loss", "step"]]

## Bibliography

- Nils Reimers and Iryna Gurevych (2019). Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks. CoRR, abs/1908.10084.
- Lingfei Wu and Ian En-Hsu Yen and Kun Xu and Fangli Xu and Avinash Balakrishnan and Pin-Yu Chen and Pradeep Ravikumar and Michael J. Witbrock (2018). Word Mover's Embedding: From Word2Vec to Document Embedding. CoRR, abs/1811.01713.
- Maas, A., Daly, R., Pham, P., Huang, D., Ng, A., & Potts, C. (2011). Learning Word Vectors for Sentiment Analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies (pp. 142–150). Association for Computational Linguistics.
- Hochreiter, S., & Schmidhuber, J. (1997). Long Short-Term Memory. Neural Computation, 9(8), 1735-1780.
- Junyoung Chung, Caglar Gulcehre, KyungHyun Cho, & Yoshua Bengio. (2014). Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling. 
- Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (2014). GloVe: Global Vectors for Word Representation. 
- Grave, E., Bojanowski, P., Gupta, P., Joulin, A., & Mikolov, T. (2018). Learning Word Vectors for 157 Languages. In Proceedings of the International Conference on Language Resources and Evaluation (LREC 2018).
- Tomas Mikolov, Kai Chen, Greg Corrado, & Jeffrey Dean. (2013). Efficient Estimation of Word Representations in Vector Space. 

